In [1]:
import numpy as np
import pandas as pd

### Loading in and splitting the data 

In [ ]:
data_path = ""

In [2]:
# clac
clac_data = pd.read_csv("/Users/laurenwhite/Library/CloudStorage/OneDrive-King'sCollegeLondon/Pre_doc_research/Interspeech_2025/data/prediction_data/clac_radar_features.csv", index_col=0) 
clac_data.drop(["pdate", "study_site", "height"], axis=1, inplace=True)

# common voice
common_data = pd.read_csv("/Users/laurenwhite/Library/CloudStorage/OneDrive-King'sCollegeLondon/Pre_doc_research/Interspeech_2025/data/prediction_data/common_radar_features.csv", index_col=0) 
common_data.drop(["pdate", "study_site", "height"], axis=1, inplace=True)

In [6]:
common_data.head()

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,15,61,1,17,0.0,0.0,-1.744000,0.000000,0.0,1.213840,0.0,0.000000,0.000000,-0.317282,0.000000,1.390084,2.082083,0.000000,0.000,-0.580913,0.0,-0.434505,-1.089088,0.000000,1.232193,0.350427
2,00d1d60a-15cf-481b-8264-91705b6f8d97,12,61,1,17,0.0,0.0,-2.545115,0.000000,0.0,1.212258,0.0,-1.121925,0.223337,-1.249521,0.000000,0.839457,1.530360,0.000000,0.000,0.000000,0.0,-0.945734,-0.439319,0.000000,0.551471,0.027242
3,00d1d60a-15cf-481b-8264-91705b6f8d97,6,61,1,17,0.0,0.0,-1.819151,0.000000,0.0,1.203555,0.0,-0.843752,-0.296925,-0.295728,0.578759,1.442836,1.809036,0.000000,-0.001,0.000000,0.0,-0.726845,-0.725703,0.000000,0.000000,0.000000
4,00d1d60a-15cf-481b-8264-91705b6f8d97,9,61,1,17,0.0,0.0,-2.176398,0.000000,0.0,1.200562,0.0,-0.616678,-0.338899,-0.712186,0.521348,1.808139,2.494563,0.553426,0.000,0.000000,0.0,0.000000,-1.225893,-0.331692,0.460893,0.437590
5,00d1d60a-15cf-481b-8264-91705b6f8d97,11,61,1,17,0.0,0.0,-1.750742,0.044732,0.0,1.198447,0.0,-1.066452,0.000000,-0.454695,0.000000,0.661622,0.930780,0.000000,0.000,0.000000,0.0,-0.255992,-0.768140,0.000000,1.476744,1.562534


In [6]:
clac_data.head()

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,15,61,1,17,0.0,0.0,0.0,0.000000,0.000000,-2.439524,-0.014556,-3.866236,0.0,0.0,-0.404939,0.000652,-0.839513,-0.838916,1.382339,4.543032,0.000000,0.000000,-1.957449,0.000000,-0.947449,-2.079705,0.000000,2.782305,0.950200
2,00d1d60a-15cf-481b-8264-91705b6f8d97,12,61,1,17,0.0,0.0,0.0,0.000000,0.000000,-3.580458,-0.642018,-3.257788,0.0,0.0,-2.054069,0.634049,-1.694526,0.000000,0.725188,3.690534,0.000000,0.000000,-0.659081,0.564567,-1.505552,-1.317413,0.000000,1.783904,0.510687
3,00d1d60a-15cf-481b-8264-91705b6f8d97,6,61,1,17,0.0,0.0,0.0,0.000000,-0.119877,-2.546552,0.000000,-2.302101,0.0,0.0,-1.736427,-0.882347,-0.819745,0.999972,1.445295,4.121132,0.087793,-0.695228,-1.058550,0.000000,-1.266593,-1.653392,-0.150280,0.823852,0.362275
4,00d1d60a-15cf-481b-8264-91705b6f8d97,9,61,1,17,0.0,0.0,0.0,0.293384,0.000000,-3.055337,0.000000,-3.237042,0.0,0.0,-1.477134,-0.940911,-1.201704,0.914478,1.881269,5.180379,0.900959,0.000000,-0.867849,0.251918,0.000000,-2.240201,-0.533363,1.651056,1.068737
5,00d1d60a-15cf-481b-8264-91705b6f8d97,11,61,1,17,0.0,0.0,0.0,0.000000,0.000000,-2.449125,0.051457,-1.983721,0.0,0.0,-1.990725,0.000000,-0.965543,0.000000,0.512950,2.764089,0.128217,0.000000,-0.977499,0.000000,-0.752567,-1.703177,0.000000,3.140984,2.598593


### Create datasets for the two model versions

In [7]:
radar_raw = pd.read_csv("/Users/laurenwhite/Library/CloudStorage/OneDrive-King'sCollegeLondon/Pre_doc_research/Interspeech_2025/data/radar/radar_data.csv", index_col=0)
radar_raw.drop(["pdate", "study_site", "height"], axis=1, inplace=True)

In [8]:
# demographics only - i know these should be the same across datasets but I want to be safe
demographics = radar_raw.loc[:,["pid", "age", "education_years", "gender", "phq8"]]

In [9]:
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error, explained_variance_score

In [10]:
import sklearn
print(sklearn.__version__)
import sys
print(sys.version)

1.5.1
3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:25:12) [Clang 14.0.6 ]
1.5.1
3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:25:12) [Clang 14.0.6 ]


In [11]:
# Custom RMSE scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Custom calibration slope scorer
def calibration_slope(y_true, y_pred):
    model = ElasticNet()
    model.fit(y_pred.reshape(-1, 1), y_true)
    return model.coef_[0]

# Custom calibration intercept scorer
def calibration_intercept(y_true, y_pred):
    model = ElasticNet()
    model.fit(y_pred.reshape(-1, 1), y_true)
    return model.intercept_

# Define the scorers dictionary
scorers = {
    'rmse': make_scorer(rmse), # NC:I removed greater_is_better=False, this only needs setting if using scorer in a loss function 
    'r2': make_scorer(r2_score),
    'mae': make_scorer(mean_absolute_error),
    'explained_variance': make_scorer(explained_variance_score),
    'calibration_slope': make_scorer(calibration_slope),
    'calibration_intercept': make_scorer(calibration_intercept)
}

In [12]:
# cv splits
inner_cv = GroupKFold(n_splits=10)
outer_cv = GroupKFold(n_splits=10)

In [13]:
# Pipeline
en_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(random_state=30))
])

# Define hyperparameters to search
en_param_grid = {
    'elasticnet__alpha': [0.1, 1.0, 10.0, 100],
    'elasticnet__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

### Demographics only

In [14]:
features = ["age", "gender", "education_years"]

X = radar_raw[features]
y = radar_raw[["phq8"]]
groups = radar_raw[["pid"]]

In [15]:
en_Inner_Grid_1 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)

In [16]:
en_demo = cross_validate(en_Inner_Grid_1, X, y, 
                         cv=outer_cv,
                         groups=groups,
                         params={'groups': groups}, 
                         scoring=scorers,
                         n_jobs = -1,
                         return_train_score=True,
                         return_estimator=True)

#### Conventional + demographics

In [17]:
features = radar_raw.columns.drop(["pid", "phq8"])

X = radar_raw[features]
y = radar_raw[["phq8"]]
groups = radar_raw[["pid"]]

In [18]:
en_Inner_Grid_2 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)

In [19]:
en_raw = cross_validate(en_Inner_Grid_2, X, y, 
                         cv=outer_cv,
                         groups=groups,
                         params={'groups': groups}, 
                         scoring=scorers,
                         n_jobs = -1,
                         return_train_score=True,
                         return_estimator=True)

### CLAC + demographics

In [21]:
en_Inner_Grid_3 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)


In [22]:
features = clac_data.columns.drop(["pid", "phq8"])

X = clac_data[features]
y = clac_data[["phq8"]]
groups = clac_data[["pid"]]

In [23]:
clac_model = cross_validate(en_Inner_Grid_3, X, y, 
                           cv=outer_cv,
                           groups=groups,
                           params={'groups': groups}, 
                           scoring=scorers,
                           return_train_score=True,
                           return_estimator=True,
                           n_jobs = -1)

### Common voice + demographics

In [24]:
en_Inner_Grid_4 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)

In [25]:
features = common_data.columns.drop(["pid", "phq8"])

X = common_data[features]
y = common_data[["phq8"]]
groups = common_data[["pid"]]

In [27]:
common_model = cross_validate(en_Inner_Grid_4, X, y, 
                           cv=outer_cv,
                           groups=groups,
                           params={'groups': groups}, 
                           scoring=scorers,
                           return_train_score=True,
                           return_estimator=True,
                           n_jobs = -1)

## Models without demographics

#### Conventional speech features

In [32]:
features = radar_raw.columns.drop(["pid", "phq8", "gender", "education_years", "age"])

X = radar_raw[features]
y = radar_raw[["phq8"]]
groups = radar_raw[["pid"]]

In [33]:
en_Inner_Grid_5 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)

In [34]:
conventional_nodemo = cross_validate(en_Inner_Grid_5, X, y, 
                         cv=outer_cv,
                         groups=groups,
                         params={'groups': groups}, 
                         scoring=scorers,
                         n_jobs = -1,
                         return_train_score=True,
                         return_estimator=True)

### CLAC

In [35]:
en_Inner_Grid_6 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)


In [36]:
features = clac_data.columns.drop(["pid", "phq8", "gender", "education_years", "age"])

X = clac_data[features]
y = clac_data[["phq8"]]
groups = clac_data[["pid"]]

In [38]:
clac_model_nodemo = cross_validate(en_Inner_Grid_6, X, y, 
                           cv=outer_cv,
                           groups=groups,
                           params={'groups': groups}, 
                           scoring=scorers,
                           return_train_score=True,
                           return_estimator=True,
                           n_jobs = -1)

### Common voice

In [39]:
en_Inner_Grid_7 = GridSearchCV(en_pipeline,
                               en_param_grid,
                               verbose = 1,
                               cv=inner_cv,
                               n_jobs = -1)

In [40]:
features = common_data.columns.drop(["pid", "phq8", "gender", "education_years", "age"])

X = common_data[features]
y = common_data[["phq8"]]
groups = common_data[["pid"]]

In [41]:
common_model_nodemo = cross_validate(en_Inner_Grid_7, X, y, 
                           cv=outer_cv,
                           groups=groups,
                           params={'groups': groups}, 
                           scoring=scorers,
                           return_train_score=True,
                           return_estimator=True,
                           n_jobs = -1)

### Results

In [42]:
current_model = en_demo
print("outer")
print(f"test_rmse: {current_model['test_rmse'].mean()}")
print(f"test_r2: {current_model['test_r2'].mean()}")
print(f"test_mae: {current_model['test_mae'].mean()}")
print("---------------------------------")
print("best params")
print(current_model['estimator'][0].best_params_)
print(current_model['estimator'][1].best_params_)
print(current_model['estimator'][2].best_params_)
print(current_model['estimator'][3].best_params_)
print(current_model['estimator'][4].best_params_)

outer
test_rmse: 5.968223083411155
test_r2: 0.011179667562676632
test_mae: 4.927800691231314
---------------------------------
best params
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
outer
test_rmse: 5.968223083411155
test_r2: 0.011179667562676632
test_mae: 4.927800691231314
---------------------------------
best params
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9}


In [44]:
results = {
    "metric": ["test_rmse", "test_r2", "test_mae"],
    "demographics_only":  [en_demo['test_rmse'].mean(), en_demo['test_r2'].mean(), en_demo['test_mae'].mean()],
    "conventional_dem":  [en_raw['test_rmse'].mean(), en_raw['test_r2'].mean(), en_raw['test_mae'].mean()],
    "clac_dem":  [clac_model['test_rmse'].mean(), clac_model['test_r2'].mean(), clac_model['test_mae'].mean()],
    "common_dem":  [common_model['test_rmse'].mean(), common_model['test_r2'].mean(), common_model['test_mae'].mean()],
    
    "conventional":  [conventional_nodemo['test_rmse'].mean(), conventional_nodemo['test_r2'].mean(), conventional_nodemo['test_mae'].mean()],
    "clac":  [clac_model_nodemo['test_rmse'].mean(), clac_model_nodemo['test_r2'].mean(), clac_model_nodemo['test_mae'].mean()],
    "common":  [common_model_nodemo['test_rmse'].mean(), common_model_nodemo['test_r2'].mean(), common_model_nodemo['test_mae'].mean()]
}

results_df = pd.DataFrame(results).T
results_df.columns = results_df.iloc[0]  # Set the first row as column names
results_df = results_df[1:]  # Remove the first row
results_df

metric,test_rmse,test_r2,test_mae
demographics_only,5.968223,0.01118,4.927801
conventional_dem,5.830419,0.053458,4.773684
clac_dem,5.855152,0.046588,4.806979
common_dem,5.84828,0.048683,4.807562
conventional,5.979893,0.008461,4.981262
clac,5.968163,0.011998,4.960459
common,5.97915,0.00838,4.976235
